This notebook is to prepare and pre-process data for various prediction models from the Used Car Price data 
at : https://www.kaggle.com/CooperUnion/cardataset, after the initial Data exploration, as given in ..... 
The numerical variables are scaled with StandardScaler, imputation strategy is used to replace 0 values with mean
StratifiedshuffleSplit is done based on Age of car (Curr Year - Year of Car), by creating Age category (Age / 5), and 
putting the values in different Age category buckets. The same distribution is maintained in Train and Test data.
The categorical variables (Transmission type, Vehicle Size and Drive wheels) are one-hot encoded and added to the feature vector. The numerical variables considered are : Age, City mpg, Engine Cylinders. Two sets of X features are 
produced, with Make and without Make and results compared.


In [2]:
#Import all necessary libraries
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

Read the pickle files prepared by stratifying the Car Sales Data based on Make and Price. This is necessary since
cars belong to different price segments, and including all make and models is not a viable solution, as the range of price
is different for the same features for different makes. The stratification details can be found in the 
Data exploration notebook, which precedes this and outputs the data into .pkl files, based on car segment/price category

In [3]:
df_ordinary=pd.read_pickle('C:/users/hackuser1/Hackathon18/ordinarydf.pkl')
df_deluxe=pd.read_pickle('C:/users/hackuser1/Hackathon18/del.pkl')
df_supdel=pd.read_pickle('C:/users/hackuser1/Hackathon18/supdel.pkl')
df_luxury=pd.read_pickle('C:/users/hackuser1/Hackathon18/luxury.pkl')
df_suplux=pd.read_pickle('C:/users/hackuser1/Hackathon18/suplux.pkl')

In [4]:
df_ordinary.head()


,Make,Model,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Vehicle Size,Vehicle Style,MSRP,Age,log_MSRP,log_city mpg
0,FIAT,124 Spider,premium unleaded (recommended),160.0,4.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,27495,0,10.221796,3.295837
1,FIAT,124 Spider,premium unleaded (recommended),160.0,4.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,24995,0,10.126471,3.295837
2,FIAT,124 Spider,premium unleaded (recommended),160.0,4.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28195,0,10.246935,3.295837
3,Chrysler,200,flex-fuel (unleaded/E85),184.0,4.0,AUTOMATIC,front wheel drive,4.0,Midsize,Sedan,25170,2,10.133448,3.178054
4,Chrysler,200,flex-fuel (unleaded/E85),184.0,4.0,AUTOMATIC,front wheel drive,4.0,Midsize,Sedan,23950,2,10.083765,3.178054


In [4]:
df_ordinary["Make"].value_counts()

Chevrolet     1123
Ford           879
Volkswagen     809
Dodge          626
Nissan         556
Honda          449
Mazda          423
Suzuki         351
Hyundai        303
Subaru         256
Kia            231
Mitsubishi     213
Chrysler       187
Pontiac        186
Oldsmobile     150
Plymouth        82
FIAT            62
Scion           60
Name: Make, dtype: int64

In [5]:
print(len(df_ordinary))
print(len(df_deluxe))
print(len(df_supdel))
print(len(df_luxury))
print(len(df_suplux))

6946
4372
52
521
19


We will take the ordinary segment as it has the most data, and fit our model. Once done the same model will be applied
to other segments.(For MVP, in actuality the modeling exercise needs to be repeated for each segment, as the relationships 
may be different)

In [5]:
df_ordinary["Number of Doors"] = df_ordinary["Number of Doors"].replace("?",0)
df_ordinary["Number of Doors"] = df_ordinary["Number of Doors"].astype('float32')
df_ordinary["MSRP"] = df_ordinary["MSRP"].replace("?",0)
df_ordinary["MSRP"] = df_ordinary["MSRP"].astype("float32")
df_ordinary["Engine HP"] = df_ordinary["Engine HP"].replace("?",0)
df_ordinary["Engine HP"] = df_ordinary["Engine HP"].astype("float32")

We check the distribution of Car Sales on the basis of Age of Car, and create Age-cat and check the distribution of the Car data based on Age-cat (Age / 5). We plan to use StratifiedSampling to make sure both Test and Train data represents same distribution of cars based on Age of Car

In [6]:
df_ordinary["Age"].value_counts()
#create a field Age-cat to divide the data into 5 Age categories, based on the Age of the car
df_ordinary["Age-cat"] = np.ceil(df_ordinary["Age"] / 5)
df_ordinary["Age-cat"].where(df_ordinary["Age-cat"] < 5, 5.0, inplace=True)
#check distribution of Age Cat in the original data
df_ordinary["Age-cat"].value_counts() / len(df_ordinary)

1.0    0.436798
2.0    0.157213
0.0    0.127987
3.0    0.105960
5.0    0.100489
4.0    0.071552
Name: Age-cat, dtype: float64

In [ ]:
We treat Engine Cylinders, Engine Fuel Type, Transmission Type, Driven_wheels, Vehicle Size and Make as 
Categorical variables based on our Data exploration. We use the LabelBinarizer to fit the variables on the entire 
set. The actual encoding will be done using the encoded values on the Train and test samples

In [8]:
car_eng_cyl = df_ordinary["Engine Cylinders"]
encoder_cyl = LabelBinarizer()
encoder_cyl.fit(car_eng_cyl)
print(encoder_cyl.classes_)

car_eng_fuel_type = df_ordinary["Engine Fuel Type"]
encoder_fuel = LabelBinarizer()
encoder_fuel.fit(car_eng_fuel_type)
print(encoder_fuel.classes_)

car_trans_type = df_ordinary["Transmission Type"]
encoder_trans = LabelBinarizer()
encoder_trans.fit(car_trans_type)
print(encoder_trans.classes_)

car_driven_wheels = df_ordinary["Driven_Wheels"]
encoder_wheels = LabelBinarizer()
encoder_wheels.fit(car_driven_wheels)
print(encoder_wheels.classes_)

car_vehicle_size = df_ordinary["Vehicle Size"]
encoder_size = LabelBinarizer()
encoder_size.fit(car_vehicle_size)
print(encoder_size.classes_)

car_make =df_ordinary["Make"]
encoder_make = LabelBinarizer()
encoder_make.fit(car_make)
print(encoder_make.classes_)

[  0.   3.   4.   5.   6.   8.  10.  12.]
['diesel' 'electric' 'flex-fuel (unleaded/E85)'
 'flex-fuel (unleaded/natural gas)' 'natural gas'
 'premium unleaded (recommended)' 'premium unleaded (required)'
 'regular unleaded']
['AUTOMATED_MANUAL' 'AUTOMATIC' 'DIRECT_DRIVE' 'MANUAL' 'UNKNOWN']
['all wheel drive' 'four wheel drive' 'front wheel drive'
 'rear wheel drive']
['Compact' 'Large' 'Midsize']
['Chevrolet' 'Chrysler' 'Dodge' 'FIAT' 'Ford' 'Honda' 'Hyundai' 'Kia'
 'Mazda' 'Mitsubishi' 'Nissan' 'Oldsmobile' 'Plymouth' 'Pontiac' 'Scion'
 'Subaru' 'Suzuki' 'Volkswagen']


In [9]:
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

for train_index, test_index in split.split(df_ordinary,df_ordinary["Age-cat"]):
    strat_train_set = df_ordinary.iloc[train_index]
    strat_test_set = df_ordinary.iloc[test_index]

In [10]:
#check distribution of Age Cat in the train data
strat_train_set["Age-cat"].value_counts() / len(strat_train_set)

1.0    0.436825
2.0    0.157127
0.0    0.127970
3.0    0.106012
5.0    0.100432
4.0    0.071634
Name: Age-cat, dtype: float64

In [11]:
#check distribution of Age Cat in the test data
strat_test_set["Age-cat"].value_counts() / len(strat_test_set)

1.0    0.436691
2.0    0.157554
0.0    0.128058
3.0    0.105755
5.0    0.100719
4.0    0.071223
Name: Age-cat, dtype: float64

Create the X and Y variables from the Feature analysis done in Exploration notebook. Repeat the same operations 
for Train and Test data.

In [12]:
carSales_X = strat_train_set.copy()
carSales_X = strat_train_set.drop("MSRP", axis=1) # drop labels for training set
carSales_X = strat_train_set.drop("log_MSRP", axis=1) # drop labels for training set
carSales_Y = strat_train_set["log_MSRP"].copy() # use log MSRP as labels for training set, based on data Exploration
carSales_Y_orig = strat_train_set["MSRP"].copy() # use MSRP as labels also for training set, to compare fit based on Log and original Price

carSales_test_X = strat_test_set.copy()
carSales_test_X = strat_test_set.drop("MSRP", axis=1) # drop labels for test set
carSales_test_X = strat_test_set.drop("log_MSRP", axis=1) # drop labels for test set
carSales_test_Y = strat_test_set["log_MSRP"].copy()# use log MSRP as labels for test set, based on data Exploration
carSales_test_Y_orig = strat_test_set["MSRP"].copy()

In [13]:
carSales_Y = carSales_Y.reshape(carSales_Y.shape[0],1)
carSales_test_Y = carSales_test_Y.reshape(carSales_test_Y.shape[0],1)
carSales_Y_orig = carSales_Y_orig.reshape(carSales_Y_orig.shape[0],1)
carSales_test_Y_orig = carSales_test_Y_orig.reshape(carSales_test_Y.shape[0],1)
print(carSales_X.shape)
print(carSales_Y.shape)
print(carSales_Y_orig.shape)
print(carSales_test_X.shape)
print(carSales_test_Y.shape)
print(carSales_test_Y_orig.shape)

(5556, 14)
(5556, 1)
(5556, 1)
(1390, 14)
(1390, 1)
(1390, 1)


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


We have 5556 rows in Train data, and 1390 rows in Test data. Now we need to remove unnecessary columns based on
Correlation analysis done in ExplorationNotebook, and do Encoding of Categorical variables. Also 
we need to do StandardNormalization before applying Regression models.

In [14]:
carSales_X.head()

,Make,Model,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Vehicle Size,Vehicle Style,MSRP,Age,log_city mpg,Age-cat
2668,Ford,F-150,regular unleaded,325.0,6.0,AUTOMATIC,rear wheel drive,4.0,Large,Crew Cab Pickup,34165.0,0,2.995732,0.0
1624,Chevrolet,Corvette,premium unleaded (recommended),460.0,8.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,70195.0,0,2.833213,0.0
4609,Nissan,Pathfinder,regular unleaded,260.0,6.0,AUTOMATIC,front wheel drive,4.0,Large,4dr SUV,36210.0,2,3.044522,1.0
3705,Volkswagen,Jetta SportWagen,diesel,140.0,4.0,AUTOMATED_MANUAL,front wheel drive,4.0,Compact,Wagon,29465.0,3,3.401197,1.0
2048,Dodge,Dynasty,regular unleaded,141.0,6.0,AUTOMATIC,front wheel drive,4.0,Midsize,Sedan,2000.0,25,2.944439,5.0


We drop all categorical columns after making a copy, and retain only the numerical features of significance

In [15]:
carSales_X_num = carSales_X
carSales_X_num  = carSales_X_num.drop("Make",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Model",axis=1)
#carSales_X_num  = carSales_X_num.drop("Year",axis=1)
carSales_X_num  = carSales_X_num.drop("Engine Cylinders",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Engine Fuel Type",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Transmission Type",axis=1) # to be treated as categorical var 
carSales_X_num  = carSales_X_num.drop("Driven_Wheels",axis=1) # to be treated as categorical var
carSales_X_num = carSales_X_num.drop("Number of Doors",axis=1) # to be treated as categorical var
#carSales_X_num  = carSales_X_num.drop("Market Category",axis=1)
carSales_X_num  = carSales_X_num.drop("Vehicle Style",axis=1)
carSales_X_num = carSales_X_num.drop("Vehicle Size",axis=1) # to be treated as categorical var
#carSales_X_num = carSales_X_num.drop("highway MPG",axis=1)
carSales_X_num = carSales_X_num.drop("Age-cat",axis=1)
carSales_X_num = carSales_X_num.drop("MSRP",axis=1)
#carSales_X_num = carSales_X_num.drop("MSRP_Median",axis=1)
#carSales_X_num = carSales_X_num.drop("MSRP_group",axis=1)
#carSales_X_num = carSales_X_num.drop("Engine HP",axis=1)

In [16]:
carSales_X_num.head()

,Engine HP,Age,log_city mpg
2668,325.0,0,2.995732
1624,460.0,0,2.833213
4609,260.0,2,3.044522
3705,140.0,3,3.401197
2048,141.0,25,2.944439


In [17]:
#Apply the same transformation on Test data
carSales_test_X_num = carSales_test_X
carSales_test_X_num  = carSales_test_X_num.drop("Make",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Model",axis=1)
#carSales_test_X_num  = carSales_test_X_num.drop("Year",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Engine Cylinders",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Engine Fuel Type",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Transmission Type",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Driven_Wheels",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Number of Doors",axis=1)
#carSales_test_X_num  = carSales_test_X_num.drop("Market Category",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Vehicle Style",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Vehicle Size",axis=1)
#carSales_test_X_num = carSales_test_X_num.drop("highway MPG",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Age-cat",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("MSRP",axis=1)
#carSales_test_X_num = carSales_test_X_num.drop("MSRP_group",axis=1)
#carSales_test_X_num = carSales_test_X_num.drop("Engine HP",axis=1)

In [18]:
carSales_test_X_num.head()

,Engine HP,Age,log_city mpg
548,170.0,13,2.944439
436,185.0,1,3.178054
6112,172.0,10,2.890372
6749,150.0,19,2.944439
3190,210.0,2,3.258097


In [19]:
#carSales_X_num["Engine HP"] = carSales_X_num["Engine HP"].astype("float32")
#carSales_X_num["Engine Cylinders"] = carSales_X_num["Engine Cylinders"].astype("float32")
#carSales_X_num["city mpg"] = carSales_X_num["city mpg"].astype("float32")
carSales_X_num["Age"] = carSales_X_num["Age"].astype("float32")
carSales_X_num.replace('null',np.NaN,inplace=True)
carSales_X_num = pd.DataFrame(carSales_X_num)
carSales_X_num = carSales_X_num.replace('?',0)
carSales_X_num = carSales_X_num.replace('NaN',0)
carSales_X_num = carSales_X_num.replace(np.NaN,0)

#carSales_test_X_num["Engine HP"] = carSales_test_X_num["Engine HP"].astype("float32")
#carSales_test_X_num["Engine Cylinders"] = carSales_test_X_num["Engine Cylinders"].astype("float32")
#carSales_test_X_num["city mpg"] = carSales_test_X_num["city mpg"].astype("float32")
carSales_test_X_num["Age"] = carSales_test_X_num["Age"].astype("float32")
carSales_test_X_num.replace('null',np.NaN,inplace=True)
carSales_test_X_num = pd.DataFrame(carSales_test_X_num)
carSales_test_X_num = carSales_test_X_num.replace('?',0)
carSales_test_X_num = carSales_test_X_num.replace('NaN',0)
carSales_test_X_num = carSales_test_X_num.replace(np.NaN,0)

In [20]:
m=carSales_X_num.isnull().any()
print(m[m])
m=np.isfinite(carSales_X_num.select_dtypes(include=['float64'])).any()
print(m[m])
m=carSales_test_X_num.isnull().any()
print(m[m])
m=np.isfinite(carSales_test_X_num.select_dtypes(include=['float64'])).any()
print(m[m])

Series([], dtype: bool)
Engine HP       True
Age             True
log_city mpg    True
dtype: bool
Series([], dtype: bool)
Engine HP       True
Age             True
log_city mpg    True
dtype: bool


Wherever there are 0 values, we replace by the mean 

In [21]:
imputer = Imputer(missing_values=0,strategy="mean")
imputer.fit(carSales_X_num)
imputer.fit(carSales_test_X_num)

Imputer(axis=0, copy=True, missing_values=0, strategy='mean', verbose=0)

In [22]:
#Standardize the data using sklearn StandardScaler
scaler = StandardScaler()
train_X = scaler.fit_transform(carSales_X_num)
test_X = scaler.transform(carSales_test_X_num)
print(train_X.shape)

(5556, 3)


Now add the Categorical variables using one-hot represenation, using the encoder already fit on the entire sample

In [23]:
car_eng_cyl = carSales_X["Engine Cylinders"]
car_eng_1hot = encoder_cyl.transform(car_eng_cyl)
print(car_eng_1hot.shape)

train_X = np.concatenate((train_X,car_eng_1hot),axis=1)

car_eng_fuel_type = carSales_X["Engine Fuel Type"]
car_fuel_1hot = encoder_fuel.transform(car_eng_fuel_type)
print(car_fuel_1hot.shape)

train_X = np.concatenate((train_X,car_fuel_1hot),axis=1)

car_trans_type = carSales_X["Transmission Type"]
car_trans_1hot = encoder_trans.transform(car_trans_type)
print(car_trans_1hot.shape)

train_X = np.concatenate((train_X,car_trans_1hot),axis=1)

car_driven_wheels = carSales_X["Driven_Wheels"]
car_drive_1hot = encoder_wheels.transform(car_driven_wheels)
print(car_drive_1hot.shape)

train_X = np.concatenate((train_X,car_drive_1hot),axis=1)

car_vehicle_size = carSales_X["Vehicle Size"]
car_size_1hot = encoder_size.transform(car_vehicle_size)
print(car_size_1hot.shape)

train_X = np.concatenate((train_X,car_size_1hot),axis=1)

car_make = carSales_X["Make"]
car_make_1hot = encoder_make.transform(car_make)
print(car_make_1hot.shape)

train_X_make = np.concatenate((train_X,car_make_1hot),axis=1)
print(train_X.shape)
print(train_X_make.shape)


(5556, 8)
(5556, 8)
(5556, 5)
(5556, 4)
(5556, 3)
(5556, 18)
(5556, 31)
(5556, 49)


In [24]:
car_eng_cyl = carSales_test_X["Engine Cylinders"]
car_eng_1hot = encoder_cyl.transform(car_eng_cyl)
print(car_eng_1hot.shape)

test_X = np.concatenate((test_X,car_eng_1hot),axis=1)

car_eng_fuel_type = carSales_test_X["Engine Fuel Type"]
car_fuel_1hot = encoder_fuel.transform(car_eng_fuel_type)
print(car_fuel_1hot.shape)

test_X = np.concatenate((test_X,car_fuel_1hot),axis=1)

car_trans_type_test = carSales_test_X["Transmission Type"]
car_trans_1hot_test = encoder_trans.transform(car_trans_type_test)
print(car_trans_1hot_test.shape)

test_X = np.concatenate((test_X,car_trans_1hot_test),axis=1)

car_driven_wheels_test = carSales_test_X["Driven_Wheels"]
car_drive_1hot_test = encoder_wheels.transform(car_driven_wheels_test)
print(car_drive_1hot_test.shape)

test_X = np.concatenate((test_X,car_drive_1hot_test),axis=1)

car_vehicle_size_test = carSales_test_X["Vehicle Size"]
car_size_1hot_test = encoder_size.transform(car_vehicle_size_test)
print(car_size_1hot_test.shape)

test_X = np.concatenate((test_X,car_size_1hot_test),axis=1)

car_make_test = carSales_test_X["Make"]
car_make_1hot_test = encoder_make.transform(car_make_test)
print(car_make_1hot_test.shape)

test_X_make = np.concatenate((test_X,car_make_1hot_test),axis=1)

print(test_X.shape)
print(test_X_make.shape)

(1390, 8)
(1390, 8)
(1390, 5)
(1390, 4)
(1390, 3)
(1390, 18)
(1390, 31)
(1390, 49)


In [25]:
train_Y = pd.DataFrame(carSales_Y)
m=train_Y.isnull().any()
print(m[m])
m=np.isfinite(train_Y.select_dtypes(include=['float64'])).any()
print(m[m])

train_Y_orig = pd.DataFrame(carSales_Y_orig)
m=train_Y_orig.isnull().any()
print(m[m])
m=np.isfinite(train_Y_orig.select_dtypes(include=['float64'])).any()
print(m[m])

test_Y = pd.DataFrame(carSales_test_Y)
m=test_Y.isnull().any()
print(m[m])
m=np.isfinite(test_Y.select_dtypes(include=['float64'])).any()
print(m[m])

test_Y_orig = pd.DataFrame(carSales_test_Y_orig)
m=test_Y_orig.isnull().any()
print(m[m])
m=np.isfinite(test_Y_orig.select_dtypes(include=['float64'])).any()
print(m[m])

Series([], dtype: bool)
0    True
dtype: bool
Series([], dtype: bool)
Series([], dtype: bool)
Series([], dtype: bool)
0    True
dtype: bool
Series([], dtype: bool)
Series([], dtype: bool)


We now take backup of the pre-processed data, so the modeling can be done instantaneously on the pre-processed data
at any later point of time

In [27]:
train_X_ordinary='C:/users/hackuser1/Hackathon18/train_X_ord.pkl'
test_X_ordinary='C:/users/hackuser1/Hackathon18/test_X_ord.pkl'
train_Y_ordinary='C:/users/hackuser1/Hackathon18/train_Y_ord.pkl'
test_Y_ordinary='C:/users/hackuser1/Hackathon18/test_Y_ord.pkl'
train_Y_ordinary_orig='C:/users/hackuser1/Hackathon18/train_Y_ord_orig.pkl'
test_Y_ordinary_orig='C:/users/hackuser1/Hackathon18/test_Y_ord_orig.pkl'

with open(train_X_ordinary, "wb") as f:
    w = pickle.dump(train_X,f)
with open(test_X_ordinary, "wb") as f:
    w = pickle.dump(test_X,f)
with open(train_Y_ordinary, "wb") as f:
    w = pickle.dump(train_Y,f)
with open(test_Y_ordinary, "wb") as f:
    w = pickle.dump(test_Y,f)
with open(train_Y_ordinary_orig, "wb") as f:
    w = pickle.dump(train_Y_orig,f)
with open(test_Y_ordinary_orig, "wb") as f:
    w = pickle.dump(test_Y_orig,f)
    
train_X_ord_make='C:/users/hackuser1/Hackathon18/train_X_ord_make.pkl'
test_X_ord_make='C:/users/hackuser1/Hackathon18/test_X_ord_make.pkl'

with open(train_X_ord_make, "wb") as f:
    w = pickle.dump(train_X_make,f)
with open(test_X_ord_make, "wb") as f:
    w = pickle.dump(test_X_make,f)
